In [1]:
import sys
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score  
from sklearn.metrics import confusion_matrix
sys.path.append(os.path.abspath('..'))  # Ajusta la ruta según la ubicación de 'src'
from src.training.etl import EDAdataset
from src.utils.conexion import SQLConnection
from src.training.feature_engineer import PreprocesadorTexto

In [2]:
params = {"medico": "PSICOLOGÍA","fechaini": "20230101","fechafin": "20250504"}
sql_path = os.path.join("..", "sql_queries", "queries.sql")
sqlconection = SQLConnection(sql_path=sql_path, params=params)
historias_clinicas_df = sqlconection.generate_dataframe()

In [3]:
df_copy = EDAdataset(historias_clinicas_df)
df_final = df_copy.dataset_eda(historias_clinicas_df)

c:\Users\ISAIAS_QUINTERO\Desktop\Universiad\MLOps\Proyecto_Final\src\training\etl.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[column_name] = self.df[column_name].str.capitalize()


In [ ]:
# feature_engineer.py

# import pandas as pd
# import re
# import spacy
# from sklearn import preprocessing

# class PreprocesadorTexto:
#     def __init__(self, df, stopwords={
#             "medico", "paciente", "psicologo", "psicologa",
#             "psicologia", "psicoterapeuta", "psicoterapia", "refiere"}):
#         self.df = df.copy()
#         self.stopwords = stopwords
#         self.nlp = spacy.load("es_core_news_lg")
#         self.stopwords = self.nlp.Defaults.stop_words.union(self.stopwords)

#     def concatenar_columnas(self, df, columna1="subjetivo", columna2="objetivo", nueva_columna="concatenada"):
#         self.df = df
#         self.df[nueva_columna] = self.df[columna1].astype(str) + " " + self.df[columna2].astype(str)
#         return self.df

#     def expresiones_regulares(self, columna):
#         self.df[columna] = (
#             self.df[columna]
#             .astype(str)
#             .str.lower()
#             .apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'[^a-zñü ]', '', x)).strip())
#         )

#     def tokenizar(self, columna):
#         self.df[columna] = (
#             self.df[columna]
#             .astype(str)
#             .fillna("")
#             .apply(lambda x: [
#                 token.text for token in self.nlp(x)
#                 if token.text.lower() not in self.stopwords and not token.is_punct and not token.is_space
#             ])
#         )

#     def lematizar(self, columna):
#         self.df[columna] = self.df[columna].apply(
#             lambda x: [token.lemma_ for token in self.nlp(" ".join(x))] if isinstance(x, list) else []
#         )

#     def label_encodering(self, columna, nueva_columna, tipo="sexo"):
#         label_encoder = preprocessing.LabelEncoder()
#         self.df[nueva_columna] = label_encoder.fit_transform(self.df[columna])
#         mapping_df = pd.DataFrame({
#             tipo.capitalize(): label_encoder.classes_,
#             'Codigo': label_encoder.transform(label_encoder.classes_)
#         })
#         return mapping_df

#     def procesar(self, columna_texto, columna_sexo=None, columna_grupo=None):
#         self.expresiones_regulares(columna_texto)
#         self.tokenizar(columna_texto)
#         self.lematizar(columna_texto)

#         mappings = {}
#         if columna_sexo:
#             mappings["sexo"] = self.label_encodering(columna_sexo, "sexo_codificado", tipo="sexo")
#         if columna_grupo:
#             mappings["grupo"] = self.label_encodering(columna_grupo, "grupo_codificado", tipo="grupo")
#         return self.df, mappings

ImportError: cannot import name 'registry' from 'thinc' (unknown location)

In [4]:
stopwords={
            "medico", "paciente", "psicologo", "psicologa",
            "psicologia", "psicoterapeuta", "psicoterapia", "refiere"
        }

preprocesador = PreprocesadorTexto(df_final, stopwords=stopwords)
df_procesado, mapeos = preprocesador.procesar(
    columna_texto="concatenada", # Columna que contiene el texto a procesar
    columna_sexo="sexo", # Columna que contiene la variable sexo
    columna_grupo="grupo" # Columna que contiene la variable grupo
)

In [13]:
# Leer dataset historia_clinicas_procesadas.xlsx
df_final = pd.read_excel("historias_clinicas_procesadas.xlsx")

# Split de datos

In [14]:
# Dividir datos antes del vectorizador
X = df_final[["concatenada", "sexo_codificado"]]    
y = df_final["grupo_codificado"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
X_train.shape

(6866, 2)

In [16]:
X_train


,concatenada,sexo_codificado
5023,"['psiquiatra', 'ver', 'necesitar', 'medicament...",0
7585,"['refierevengo', 'tratamiento', 'año', 'enviar...",0
1598,"['dinmico', 'familiar', 'continuo', 'explicar'...",1
9162,"['expresar', 'momento', 'sentir', 'ocasión', '...",0
300,"['tener', 'problema', 'mamo', 'viernes', 'habl...",0
...,...,...
5734,"['estrado', 'depresin', 'ansiedad', 'tranquili...",0
5191,"['decir', 'cosa', 'padr', 'preocupado', 'separ...",1
5390,"['cosa', 'afectar', 'sentir', 'desbordado', 'a...",1
860,"['venir', 'situacin', 'familiar', 'venir', 'pr...",0


In [17]:
# Vectorizacion
vectorizer_LR = TfidfVectorizer()
X_train_vect = vectorizer_LR.fit_transform(X_train["concatenada"])
X_test_vect = vectorizer_LR.transform(X_test["concatenada"])

In [29]:
from sklearn.pipeline import Pipeline

# Creamos el Pipeline completo

model_pipeline = Pipeline(steps=[
    ("classifier", LogisticRegression(random_state=42, max_iter=1000))
])

In [30]:
# Fit the pipeline 
model_pipeline.fit(X_train_vect, y_train)

,steps,"[('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None


In [40]:
# Make predictions
y_pred = model_pipeline.predict(X_test_vect)
y_pred_proba = model_pipeline.predict_proba(X_test_vect)[:, 1]

In [42]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.6010873258579681
Precision: 0.6199477054971955
Recall: 0.6010873258579681
F1 Score: 0.583542767746734
Confusion Matrix:
 [[304   1 118  36  20  43]
 [ 25  16  78  26   1   2]
 [ 54   0 765 104   7  17]
 [ 49   2 229 383   3   9]
 [ 72   1  64  18  98  44]
 [ 43   0  64  35   9 203]]


# Training con MLFlow

In [43]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [44]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [45]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1758418427717, experiment_id='0', last_update_time=1758418427717, lifecycle_stage='active', name='Default', tags={}>]

## Prueba con autologgin

In [46]:
experiment_name = "nlp_logistic_regression_v1"
mlflow.set_experiment(experiment_name)
mlflow.autolog()

with mlflow.start_run():
    model_pipeline.fit(X_train_vect, y_train)
    y_pred = model_pipeline.predict(X_test_vect)
    mlflow.log_param("model_type", "Logistic Regression")
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("precision", precision_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("recall", recall_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred, average='weighted'))


2025/09/20 20:41:05 INFO mlflow.tracking.fluent: Experiment with name 'nlp_logistic_regression_v1' does not exist. Creating a new experiment.
2025/09/20 20:41:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/09/20 20:41:08 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/20 20:41:21 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run monumental-doe-781 at: http://127.0.0.1:5000/#/experiments/1/runs/ead316c92ce44272a3edb391e2805540
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [4]:
# train.py

class Train:
    def __init__(self, df, target_column, model, test_size=0.3, training_columns=["concatenada", "sexo_codificado"]):
        self.df = df
        self.target_column = target_column
        self.test_size = test_size
        self.model = model
        self.training_columns = training_columns

    def train_test_split(self):
        X_train, X_test, y_train, y_test = train_test_split(self.df[self.training_columns], self.df[self.target_column], test_size=self.test_size, random_state=42)
        return X_train, X_test, y_train, y_test
    
    def vectorizer(self, X_train, X_test, vectorizer_model=TfidfVectorizer(), columns_to_vectorize="concatenada"):
        self.vectorizer_model = vectorizer_model
        self.X_train = X_train
        self.X_test = X_test
        X_train_vect = self.vectorizer_model.fit_transform(X_train[columns_to_vectorize])
        X_test_vect = self.vectorizer_model.transform(X_test[columns_to_vectorize])
        return X_train_vect, X_test_vect

    def create_pipeline_train(self):
        pipeline = Pipeline(steps=[("classifier", self.model)])
        return pipeline

    def train(self):
        X_train, X_test, y_train, y_test = self.train_test_split()
        X_train_vect, X_test_vect = self.vectorizer(X_train=X_train, X_test=X_test)
        pipeline = self.create_pipeline_train()
        pipeline.fit(X_train_vect, y_train)
        return X_test_vect, pipeline

# Modelado sin mlflow

In [66]:
# Leer dataset historia_clinicas_procesadas.xlsx
df_final = pd.read_excel("historias_clinicas_procesadas.xlsx")

In [67]:
# sys.path.append(os.path.abspath('..'))  # Ajusta la ruta según la ubicación de 'src'
# from src.training.train import Train
from sklearn.ensemble import RandomForestClassifier

In [73]:
target_column = "grupo_codificado"
training_columns = ["concatenada", "sexo_codificado"]
test_size = 0.3
model = RandomForestClassifier(random_state=42)


In [74]:
train = Train(df_final, target_column, model, test_size, training_columns=training_columns)
X_test_vect, pipeline = train.train()

In [75]:
pipeline

,steps,"[('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'


In [76]:
X_test_vect

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 270950 stored elements and shape (2943, 25352)>

In [77]:
y_pred = pipeline.predict(X_test_vect)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5915732246007476
Precision: 0.6269865261947672
Recall: 0.5915732246007476
F1 Score: 0.5756189278950028
Confusion Matrix:
 [[269   2 167  28  12  44]
 [ 18  28  85  14   2   1]
 [ 61   1 820  45   7  13]
 [ 52   3 271 339   6   4]
 [ 59   2  79  15 106  36]
 [ 49   0  91  26   9 179]]


# Modelado con MLFlow

In [5]:
# train_with_mlflow.py

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import mlflow
import mlflow.sklearn
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score
)
import warnings
warnings.filterwarnings('ignore')


class TrainMlflow:
    def __init__(
        self, df, target_column, model, 
        test_size=0.3, training_columns=["concatenada", "sexo_codificado"], model_params=None, mlflow_setup=None
    ):
        """
        Initialize the Train class with MLflow tracking capabilities.

        Args:
            df: Input dataframe
            target_column: Name of target column
            training_columns: List of feature column names to use for training
            model: Model instance to train
            model_params: Dictionary of model parameters (optional)
            experiment_name: Name for MLflow experiment
            test_size: Proportion of data to use for testing
        """
        self.df = df
        self.target_column = target_column
        self.test_size = test_size
        self.model = model
        self.training_columns = training_columns
        self.model_params = model_params if model_params is not None else {}
   

        # Set up MLflow tracking
        self.setup = mlflow_setup
        

    def train_test_split(self):
        X = self.df[self.training_columns]
        y = self.df[self.target_column]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=42
        )
        return X_train, X_test, y_train, y_test

    def vectorizer(self, X_train, X_test, vectorizer_model=TfidfVectorizer(), columns_to_vectorize="concatenada"):
        self.vectorizer_model = vectorizer_model
        self.X_train = X_train
        self.X_test = X_test
        X_train_vect = self.vectorizer_model.fit_transform(X_train[columns_to_vectorize])
        X_test_vect = self.vectorizer_model.transform(X_test[columns_to_vectorize])
        return X_train_vect, X_test_vect


    def create_pipeline_train(self):
        pipeline = Pipeline(steps=[("classifier", self.model)])
        return pipeline

    def train(self):
        """
        Train the model with MLflow tracking enabled.

        Returns:
            pipeline: Trained sklearn pipeline
            run_id: MLflow run ID for tracking
        """
        with mlflow.start_run() as run:
            # Log parameters
            mlflow.log_param("test_size", self.test_size)
            mlflow.log_param("model_type", type(self.model).__name__)
            mlflow.log_param("target_column", self.target_column)

            # Log model parameters if provided
            for param_name, param_value in self.model_params.items():
                mlflow.log_param(f"model_{param_name}", param_value)

            # Split data
            X_train, X_test, y_train, y_test = self.train_test_split()

            # Log dataset information
            mlflow.log_param("n_train_samples", len(X_train))
            mlflow.log_param("n_test_samples", len(X_test))

            # Create and train pipeline
            pipeline = self.create_pipeline_train()
            X_train_vect, X_test_vect = self.vectorizer(X_train=X_train, X_test=X_test)

            # Fit the pipeline (autolog will capture training metrics)
            pipeline.fit(X_train_vect, y_train)

            # Make predictions and log additional metrics
            y_train_pred = pipeline.predict(X_train_vect)
            y_test_pred = pipeline.predict(X_test_vect)

            # Calculate and log metrics
            train_metrics = self._calculate_metrics(
                y_train, y_train_pred, prefix="train"
            )
            test_metrics = self._calculate_metrics(
                y_test, y_test_pred, prefix="test"
            )

            # Log metrics
            all_metrics = {**train_metrics, **test_metrics}
            for metric_name, metric_value in all_metrics.items():
                mlflow.log_metric(metric_name, metric_value)

            # Get the run ID for reference
            run_id = run.info.run_id

            print(f"MLflow Run ID: {run_id}")
            print(f"Tracking URI: {mlflow.get_tracking_uri()}")
            print(f"Train Accuracy: {train_metrics['train_accuracy']:.4f}")
            print(f"Test Accuracy: {test_metrics['test_accuracy']:.4f}")

            return pipeline, run_id

    def _calculate_metrics(self, y_true, y_pred, prefix=""):
        """
        Calculate classification metrics.

        Args:
            y_true: True labels
            y_pred: Predicted labels
            prefix: Prefix for metric names (e.g., "train" or "test")

        Returns:
            Dictionary of metrics
        """
        metrics = {}

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        metrics[f"{prefix}_accuracy"] = accuracy

        # For binary and multiclass classification
        try:
            precision = precision_score(
                y_true, y_pred, average='weighted', zero_division=0
            )
            recall = recall_score(
                y_true, y_pred, average='weighted', zero_division=0
            )
            f1 = f1_score(
                y_true, y_pred, average='weighted', zero_division=0
            )

            metrics[f"{prefix}_precision"] = precision
            metrics[f"{prefix}_recall"] = recall
            metrics[f"{prefix}_f1"] = f1
        except Exception:
            # In case of any issues with multiclass metrics
            pass

        return metrics

## Modelo con Random Forest

In [80]:
experiment_name = "nlp-random-forest"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Enable autologging for sklearn models
mlflow.sklearn.autolog(
    log_input_examples=True,
    log_model_signatures=True,
    log_models=True,
    disable=False,
    exclusive=False,
    disable_for_unsupported_versions=False,
    silent=False,
    max_tuning_runs=5
)

In [81]:
params_random_forest = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model = RandomForestClassifier(random_state=42, **params_random_forest)
target_column = "grupo_codificado"
training_columns = ["concatenada", "sexo_codificado"]
test_size = 0.3

In [83]:


train_mlflow = TrainMlflow(df_final, target_column, model, test_size, training_columns=training_columns, mlflow_setup=mlflow)
train_mlflow.train()

2025/09/21 12:25:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/21 12:25:51 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


MLflow Run ID: a539033b7572460ea2d09b18ee6e8006
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.4642
Test Accuracy: 0.4108
🏃 View run honorable-skink-30 at: http://127.0.0.1:5000/#/experiments/2/runs/a539033b7572460ea2d09b18ee6e8006
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


(Pipeline(steps=[('classifier',
                  RandomForestClassifier(max_depth=10, random_state=42))]),
 'a539033b7572460ea2d09b18ee6e8006')

## Modelo con XGB

In [85]:
from xgboost import XGBClassifier

experiment_name = "nlp-xgboost"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Enable autologging for sklearn models
mlflow.xgboost.autolog()

In [86]:
params_xgb = {
    'n_estimators': 100,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1
}
model_xgb = XGBClassifier(random_state=42, **params_xgb)

In [87]:
train_mlflow = TrainMlflow(df_final, target_column, model_xgb, test_size, training_columns=training_columns, mlflow_setup=mlflow)
train_mlflow.train()

2025/09/21 12:31:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/21 12:32:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


MLflow Run ID: 5a14e6d3e90e41b19d46bb6f0b1fb90a
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9978
Test Accuracy: 0.6069
🏃 View run victorious-asp-659 at: http://127.0.0.1:5000/#/experiments/3/runs/5a14e6d3e90e41b19d46bb6f0b1fb90a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3


(Pipeline(steps=[('classifier',
                  XGBClassifier(base_score=None, booster=None, callbacks=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, device=None,
                                early_stopping_rounds=None,
                                enable_categorical=False, eval_metric=None,
                                feature_types=None, feature_weights=None,
                                gamma=None, grow_policy=None,
                                importance_type=None,
                                interaction_constraints=None, learning_rate=None,
                                max_bin=None, max_cat_threshold=None,
                                max_cat_to_onehot=None, max_delta_step=None,
                                max_depth=10, max_leaves=None,
                                min_child_weight=None, min_samples_leaf=1,
                                min_samples_split

## Modelo con MLFlow + Optuna

In [6]:
# train_with_mlflow_optuna.py

import warnings
import mlflow
import mlflow.sklearn
import optuna
from optuna.integration.mlflow import MLflowCallback
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier  # noqa: F401
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, f1_score, precision_score,
                             recall_score, roc_auc_score)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

warnings.filterwarnings('ignore')


class TrainMlflowOptuna:
    def __init__(
        self, df, target_column, model_class=RandomForestClassifier,
        test_size=0.3, training_columns=["concatenada", "sexo_codificado"], model_params=None, mlflow_setup=None,
        n_trials=10, optimization_metric='accuracy',
        param_distributions=None
    ):
        """
        Initialize the Train class with MLflow and Optuna capabilities.

        Args:
            df: Input dataframe
            target_column: Name of target column
            model_class: Model class to instantiate (not an instance)
            training_columns: List of columns to use for training
            model_params: Dictionary of fixed model parameters (optional)
            mlflow_setup: MLflow configuration
            test_size: Proportion of data to use for testing
            n_trials: Number of Optuna trials to run
            optimization_metric: Metric to optimize ('accuracy', 'f1', 'precision', 'recall', 'roc_auc')
            param_distributions: Dictionary defining parameter search space
                Example: {
                    'C': ('float', 0.001, 100, True),  # (type, min, max, log)
                    'penalty': ('categorical', ['l1', 'l2']),
                    'max_iter': ('int', 100, 1000)
                }
        """
        self.df = df

        self.target_column = target_column
        self.test_size = test_size
        self.training_columns = training_columns
        self.model_class = model_class
        self.model_params = model_params if model_params is not None else {}
        self.n_trials = n_trials
        self.optimization_metric = optimization_metric
        self.param_distributions = param_distributions or {}

        # Set up MLflow tracking
        self.setup = mlflow_setup
        
        # Store best model and parameters
        self.best_model = None
        self.best_params = None
        self.best_pipeline = None
        self.best_score = None
        

    def train_test_split(self):
        X = self.df[self.training_columns]
        y = self.df[self.target_column]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=42
        )
        return X_train, X_test, y_train, y_test

    def vectorizer(self, X_train, X_test, vectorizer_model=TfidfVectorizer(), columns_to_vectorize="concatenada"):
        self.vectorizer_model = vectorizer_model
        self.X_train = X_train
        self.X_test = X_test
        X_train_vect = self.vectorizer_model.fit_transform(X_train[columns_to_vectorize])
        X_test_vect = self.vectorizer_model.transform(X_test[columns_to_vectorize])
        return X_train_vect, X_test_vect


    def create_pipeline_train(self, model):
        """Create pipeline with a specific model instance."""
        pipeline = Pipeline(steps=[("classifier", model)])
        return pipeline

    def create_objective(self, X_train, X_test, y_train, y_test):
        """Create an Optuna objective function for hyperparameter optimization."""
        
        def objective(trial):
            # Suggest hyperparameters from the provided distributions
            params = {}
            for param_name, param_config in self.param_distributions.items():
                param_type = param_config[0]
                
                if param_type == 'float':
                    min_val, max_val = param_config[1], param_config[2]
                    log = param_config[3] if len(param_config) > 3 else False
                    params[param_name] = trial.suggest_float(
                        param_name, min_val, max_val, log=log
                    )
                elif param_type == 'int':
                    min_val, max_val = param_config[1], param_config[2]
                    params[param_name] = trial.suggest_int(
                        param_name, min_val, max_val
                    )
                elif param_type == 'categorical':
                    choices = param_config[1]
                    params[param_name] = trial.suggest_categorical(
                        param_name, choices
                    )
            
            # Merge with fixed params if provided
            all_params = {**self.model_params, **params}
            
            # Create model instance with suggested parameters
            model = self.model_class(**all_params)
            
            # Create and train pipeline
            pipeline = self.create_pipeline_train(model)
            X_train_vect, X_test_vect = self.vectorizer(X_train=X_train, X_test=X_test)

            
            # Fit the pipeline
            pipeline.fit(X_train_vect, y_train)
            
            # Calculate the optimization metric
            if self.optimization_metric == 'accuracy':
                y_pred = pipeline.predict(X_test_vect)
                score = accuracy_score(y_test, y_pred)
            elif self.optimization_metric == 'f1':
                y_pred = pipeline.predict(X_test_vect)
                score = f1_score(y_test, y_pred, average='weighted')
            elif self.optimization_metric == 'precision':
                y_pred = pipeline.predict(X_test_vect)
                score = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            elif self.optimization_metric == 'recall':
                y_pred = pipeline.predict(X_test_vect)
                score = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            elif self.optimization_metric == 'roc_auc':
                # Para clasificación multiclase, usar multi_class='ovr'
                if hasattr(pipeline, 'predict_proba'):
                    y_pred_proba = pipeline.predict_proba(X_test_vect)
                    score = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
                else:
                    # Si el modelo no soporta predict_proba, usar accuracy como fallback
                    y_pred = pipeline.predict(X_test_vect)
                    score = accuracy_score(y_test, y_pred)
            else:
                raise ValueError(f"Unknown optimization metric: {self.optimization_metric}")
            
            return score
        
        return objective
    
    def train_with_optuna(self):
        """
        Train the model with Optuna hyperparameter optimization and MLflow tracking.

        Returns:
            best_pipeline: Best trained sklearn pipeline
            best_run_id: MLflow run ID for the best model
            study: Optuna study object with optimization results
        """
        # Split data once for all trials
        X_train, X_test, y_train, y_test = self.train_test_split()
        
        # Create Optuna study
        study = optuna.create_study(
            direction='maximize',
            study_name=f"optuna_{self.model_class.__name__}"
        )
        
        # Create objective function
        objective = self.create_objective(X_train, X_test, y_train, y_test)
        
        # Set up MLflow callback for Optuna
        mlflow_callback = MLflowCallback(
            tracking_uri=mlflow.get_tracking_uri(),
            metric_name=self.optimization_metric
        )
        
        # Run optimization
        print(f"Starting Optuna optimization with {self.n_trials} trials...")
        print(f"Optimizing for: {self.optimization_metric}")
        print(f"Model type: {self.model_class.__name__}")
        
        study.optimize(
            objective, 
            n_trials=self.n_trials,
            callbacks=[mlflow_callback]
        )
        
        # Get best parameters
        self.best_params = study.best_params
        self.best_score = study.best_value
        
        print(f"\nOptimization complete!")
        print(f"Best {self.optimization_metric}: {self.best_score:.4f}")
        print(f"Best parameters: {self.best_params}")
        
        # Train final model with best parameters and log to MLflow
        best_run_id = self._train_best_model(X_train, X_test, y_train, y_test)
        
        return self.best_pipeline, best_run_id, study
    
    def _train_best_model(self, X_train, X_test, y_train, y_test):
        """Train the final model with best parameters and comprehensive MLflow logging."""
        
        with mlflow.start_run(run_name=f"best_model_{self.model_class.__name__}") as run:
            # Log basic parameters
            mlflow.log_param("test_size", self.test_size)
            mlflow.log_param("model_type", self.model_class.__name__)
            mlflow.log_param("target_column", self.target_column)
            mlflow.log_param("n_trials", self.n_trials)
            mlflow.log_param("optimization_metric", self.optimization_metric)
            
            # Log best hyperparameters
            for param_name, param_value in self.best_params.items():
                mlflow.log_param(f"best_{param_name}", param_value)
            
            # Log fixed parameters if any
            for param_name, param_value in self.model_params.items():
                mlflow.log_param(f"fixed_{param_name}", param_value)
            
            # Log dataset information
            mlflow.log_param("n_train_samples", len(X_train))
            mlflow.log_param("n_test_samples", len(X_test))
            
            # Create model with best parameters
            all_params = {**self.model_params, **self.best_params}
            self.best_model = self.model_class(**all_params)
            
            # Create and train pipeline
            self.best_pipeline = self.create_pipeline_train(self.best_model)
            # Vectorize data
            X_train_vect, X_test_vect = self.vectorizer(X_train=X_train, X_test=X_test)
            # Fit the best pipeline
            self.best_pipeline.fit(X_train_vect, y_train)
            
            # Make predictions
            y_train_pred = self.best_pipeline.predict(X_train_vect)
            y_test_pred = self.best_pipeline.predict(X_test_vect)
            
            # Calculate and log metrics
            train_metrics = self._calculate_metrics(y_train, y_train_pred, prefix="train")
            test_metrics = self._calculate_metrics(y_test, y_test_pred, prefix="test")
            
            all_metrics = {**train_metrics, **test_metrics}
            for metric_name, metric_value in all_metrics.items():
                mlflow.log_metric(metric_name, metric_value)
            
            # Log the optimization score from Optuna
            mlflow.log_metric(f"optuna_best_{self.optimization_metric}", self.best_score)
            
            # Log the model
            mlflow.sklearn.log_model(
                self.best_pipeline,
                "model",
                input_example=X_train.iloc[:5],
                signature=mlflow.models.infer_signature(X_train_vect, y_train_pred)
            )
            
            # Get the run ID for reference
            run_id = run.info.run_id
            
            print(f"\nBest Model MLflow Run ID: {run_id}")
            print(f"Tracking URI: {mlflow.get_tracking_uri()}")
            print(f"Train Accuracy: {train_metrics['train_accuracy']:.4f}")
            print(f"Test Accuracy: {test_metrics['test_accuracy']:.4f}")
            
            return run_id
    
    def train(self):
        """Alias for train_with_optuna to maintain compatibility."""
        return self.train_with_optuna()

    def _calculate_metrics(self, y_true, y_pred, prefix=""):
        """
        Calculate classification metrics.

        Args:
            y_true: True labels
            y_pred: Predicted labels
            prefix: Prefix for metric names (e.g., "train" or "test")

        Returns:
            Dictionary of metrics
        """
        metrics = {}

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        metrics[f"{prefix}_accuracy"] = accuracy

        # For binary and multiclass classification
        try:
            precision = precision_score(
                y_true, y_pred, average='weighted', zero_division=0
            )
            recall = recall_score(
                y_true, y_pred, average='weighted', zero_division=0
            )
            f1 = f1_score(
                y_true, y_pred, average='weighted', zero_division=0
            )

            metrics[f"{prefix}_precision"] = precision
            metrics[f"{prefix}_recall"] = recall
            metrics[f"{prefix}_f1"] = f1
        except Exception:
            # In case of any issues with multiclass metrics
            pass

        return metrics

## Random Forest con MLFLow + Optuna

In [93]:
from sklearn.ensemble import RandomForestClassifier

experiment_name = "nlp-RandomForest-optuna"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Define RandomForest parameter distributions
param_distributions = {
    'n_estimators': ('int', 50, 200),
    'max_depth': ('int', 5, 30),
    'min_samples_split': ('int', 2, 10),
    'min_samples_leaf': ('int', 1, 5),
    'max_features': ('categorical', ['sqrt', 'log2', None])
}

# Create trainer for RandomForest
trainer = TrainMlflowOptuna(
    df=df_final,
    target_column=target_column,
    model_class=RandomForestClassifier,
    test_size=0.3,
    n_trials=30,
    optimization_metric='f1',  # Optimize for F1 score
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'n_jobs': -1},
    mlflow_setup = mlflow
)

# Run optimization
best_pipeline, run_id, study = trainer.train()

[I 2025-09-21 14:44:36,665] A new study created in memory with name: optuna_RandomForestClassifier


Starting Optuna optimization with 30 trials...
Optimizing for: f1
Model type: RandomForestClassifier


2025/09/21 14:44:37 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1674963fcd2845f89c9de23c5f98949d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/09/21 14:44:38 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/21 14:44:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run brawny-hog-200 at: http://127.0.0.1:5000/#/experiments/4/runs/1674963fcd2845f89c9de23c5f98949d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4


[I 2025-09-21 14:44:46,573] Trial 0 finished with value: 0.5100493471199063 and parameters: {'n_estimators': 101, 'max_depth': 28, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.5100493471199063.


...

🏃 View run secretive-finch-894 at: http://127.0.0.1:5000/#/experiments/5/runs/3dcd45834ade4be8919f8db91cb2b796
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


[I 2025-09-21 15:17:29,678] Trial 29 finished with value: 0.2036487333799835 and parameters: {'n_estimators': 103, 'max_depth': 28, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 24 with value: 0.5761364322204333.


🏃 View run 29 at: http://127.0.0.1:5000/#/experiments/5/runs/f98149b926c148949072c4c919171c3f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5

Optimization complete!
Best f1: 0.5761
Best parameters: {'n_estimators': 112, 'max_depth': 30, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': None}


2025/09/21 15:17:33 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/21 15:19:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/21 15:19:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/21 15:19:20 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/21 15:19:21 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "c.... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model ur


Best Model MLflow Run ID: 0b9142777e2c47f8bec5da5a1bc38664
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9857
Test Accuracy: 0.5923
🏃 View run best_model_RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/5/runs/0b9142777e2c47f8bec5da5a1bc38664
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


## SVC con MLFLow + Optuna

In [8]:
from sklearn.svm import SVC

experiment_name = "nlp-SVC-optuna"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Define SVC parameter distributions for Optuna
param_distributions = {
    'C': ('float', 0.01, 10, True),  # log scale
    'gamma': ('float', 1e-4, 1.0, True),  # log scale

}

target_column = "grupo_codificado"
training_columns = ["concatenada", "sexo_codificado"]
test_size = 0.3

trainer = TrainMlflowOptuna(
    df=df_final,
    target_column=target_column,
    model_class=SVC,
    test_size=0.3,
    n_trials=30,
    optimization_metric='f1',
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'probability': True},
    mlflow_setup=mlflow
)

best_pipeline, run_id, study = trainer.train()


[I 2025-09-21 16:06:16,494] A new study created in memory with name: optuna_SVC


Starting Optuna optimization with 30 trials...
Optimizing for: f1
Model type: SVC


[I 2025-09-21 16:10:56,973] Trial 0 finished with value: 0.15667153203060377 and parameters: {'C': 0.0782712035827393, 'gamma': 0.07033640977484107}. Best is trial 0 with value: 0.15667153203060377.


🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/7/runs/63b3bbdfcd2b4e9084454641216bd50d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


[I 2025-09-21 16:15:50,331] Trial 1 finished with value: 0.15667153203060377 and parameters: {'C': 0.46116394103450953, 'gamma': 0.0013409119274927913}. Best is trial 0 with value: 0.15667153203060377.


...

🏃 View run 28 at: http://127.0.0.1:5000/#/experiments/7/runs/89029437b1ff4ef1b2ba645b4f5e479e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


[I 2025-09-21 18:37:54,506] Trial 29 finished with value: 0.6326434457285134 and parameters: {'C': 9.476729363788678, 'gamma': 0.058874845507113034}. Best is trial 26 with value: 0.6587562297350188.


🏃 View run 29 at: http://127.0.0.1:5000/#/experiments/7/runs/21323211f3b543d79da647784de8024f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7

Optimization complete!
Best f1: 0.6588
Best parameters: {'C': 3.8026413797244305, 'gamma': 0.35151219284825047}


2025/09/21 18:44:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/21 18:44:53 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/21 18:44:53 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "c.... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: Invalid input. Data is not compatible with model signature. Failed to convert column concatenada to type 'float64'. Error: 'ValueError('could not convert stri


Best Model MLflow Run ID: 13d40a8983d945a5b99895ae69762cdb
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9703
Test Accuracy: 0.6656
🏃 View run best_model_SVC at: http://127.0.0.1:5000/#/experiments/7/runs/13d40a8983d945a5b99895ae69762cdb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


In [9]:
from xgboost import XGBClassifier

experiment_name = "nlp-XGBC-optuna"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(experiment_name)

# Define XGBClassifier parameter distributions for Optuna
param_distributions = {
    'n_estimators': ('int', 50, 200),
    'max_depth': ('int', 3, 15)
}

trainer = TrainMlflowOptuna(
    df=df_final,
    target_column=target_column,
    model_class=XGBClassifier,
    test_size=test_size,
    n_trials=20,
    optimization_metric='f1',
    param_distributions=param_distributions,
    model_params={'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'mlogloss'},
    mlflow_setup=mlflow
)

best_pipeline, run_id, study = trainer.train()


2025/09/21 18:50:40 INFO mlflow.tracking.fluent: Experiment with name 'nlp-XGBC-optuna' does not exist. Creating a new experiment.
[I 2025-09-21 18:50:40,539] A new study created in memory with name: optuna_XGBClassifier


Starting Optuna optimization with 20 trials...
Optimizing for: f1
Model type: XGBClassifier


[I 2025-09-21 18:51:57,174] Trial 0 finished with value: 0.5834686145942526 and parameters: {'n_estimators': 80, 'max_depth': 7}. Best is trial 0 with value: 0.5834686145942526.
2025/09/21 18:51:57 INFO mlflow.tracking.fluent: Experiment with name 'optuna_XGBClassifier' does not exist. Creating a new experiment.


🏃 View run 0 at: http://127.0.0.1:5000/#/experiments/9/runs/d1d900cc4da4463eb0f672aa570eb54f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9


[I 2025-09-21 18:53:36,651] Trial 1 finished with value: 0.6001384070357436 and parameters: {'n_estimators': 136, 'max_depth': 6}. Best is trial 1 with value: 0.6001384070357436.


...

🏃 View run 18 at: http://127.0.0.1:5000/#/experiments/9/runs/f858394d2306461d82b3e8efc49961d3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9


[I 2025-09-21 19:55:12,342] Trial 19 finished with value: 0.6055386725840102 and parameters: {'n_estimators': 181, 'max_depth': 10}. Best is trial 8 with value: 0.6119811691484995.


🏃 View run 19 at: http://127.0.0.1:5000/#/experiments/9/runs/23a42d52b9b64b9b9fadfa4560385a57
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9

Optimization complete!
Best f1: 0.6120
Best parameters: {'n_estimators': 183, 'max_depth': 12}


2025/09/21 19:59:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/21 19:59:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/21 19:59:39 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "c.... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: Invalid input. Data is not compatible with model signature. Failed to convert column concatenada to type 'float64'. Error: 'ValueError('could not convert stri


Best Model MLflow Run ID: 20057cb41afe40329bd1ea9aac701182
Tracking URI: http://127.0.0.1:5000
Train Accuracy: 0.9978
Test Accuracy: 0.6215
🏃 View run best_model_XGBClassifier at: http://127.0.0.1:5000/#/experiments/9/runs/20057cb41afe40329bd1ea9aac701182
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9
